### Visualization of the electrode coverage using MNE tools

In [1]:
import numpy as np
import mne
import pandas as pd
import os
import mne_bids
import nibabel


mne.viz.set_3d_backend('notebook')
%matplotlib qt

Using notebook 3d backend.



### Specify info about the subject, recording session, datatype, acquisition, task and freesurfer reconstruction directory

Most of this info can be derived from the BIDs filenames and metadata. Check https://bids-specification.readthedocs.io/en/stable/04-modality-specific-files/04-intracranial-electroencephalography.html

To visualize electrode coverage on the native brain you will need to obtain freesurfer reconstruction of the native anatomy. For this, you will need to install Freesurfer (https://surfer.nmr.mgh.harvard.edu/) and run the following in the terminal, set up variables SUBJECTS_DIR and FS_HOME_DIR and run cortical reconstruction in the terminal: 

``` recon-all -subject sub-01  -i sub-01/ses-mri3t/anat/sub-01_ses-mri3t_run-1_T1w.nii -cw256 -all ```

In [2]:
bids_dir='/Fridge/users/julia/project_chill_dataset_paper/data/BIDS2'
subjects = mne_bids.get_entity_vals(bids_dir, 'subject')

subject = '01'
session = 'iemu'
datatype = 'ieeg'
task = 'film'
acquisition = 'clinical'
fs_dir = '/Fridge/users/julia/project_chill_dataset_paper/data/freesurfer2/sub-01'

### Load electrodes info

In [3]:
electrodes_path = mne_bids.BIDSPath(subject=subject,
                                    session=session,
                                    suffix='electrodes',
                                    extension='tsv',
                                    datatype=datatype,
                                    acquisition=acquisition,
                                    root=bids_dir)
electrodes = pd.read_csv(str(electrodes_path), sep='\t', header=0, index_col=None)
coords = electrodes[['x', 'y', 'z']].values

### Load channels info

In [4]:
channels_path = mne_bids.BIDSPath(subject=subject,
                                    session=session,
                                    suffix='channels',
                                    extension='tsv',
                                    datatype=datatype,
                                    task=task,
                                    acquisition=acquisition,
                                    root=bids_dir)
channels = pd.read_csv(str(channels_path.match()[0]), sep='\t', header=0, index_col=None)

### Load iEEG data info, set channel types and drop all irrelevant channels (not iEEG)

In [5]:
data_path = mne_bids.BIDSPath(subject=subject,
                                    session=session,
                                    suffix='ieeg',
                                    extension='vhdr',
                                    datatype=datatype,
                                    task=task,
                                    acquisition=acquisition,
                                    root=bids_dir)
raw = mne.io.read_raw_brainvision(str(data_path.match()[0]), scale=1.0, preload=False, verbose=True)
raw.set_channel_types({ch_name: str(x).lower()
                if str(x).lower() in ['ecog', 'seeg'] else 'misc'
                                for ch_name, x in zip(raw.ch_names, channels['type'].values)})
raw.drop_channels([raw.ch_names[i] for i, j in enumerate(raw.get_channel_types()) if j == 'misc'])

Extracting parameters from /Fridge/users/julia/project_chill_dataset_paper/data/BIDS2/sub-01/ses-iemu/ieeg/sub-01_ses-iemu_task-film_acq-clinical_run-1_ieeg.vhdr...
Setting channel info structure...


<RawBrainVision | sub-01_ses-iemu_task-film_acq-clinical_run-1_ieeg.eeg, 103 x 860254 (420.0 s), ~107 kB, data not loaded>

### Transform electrode coordinate to the freesurfer RAS space to visualize them on the surface

In [6]:
x = nibabel.load(os.path.join(fs_dir, 'mri', 'orig.mgz'))
vox_coords = np.round(mne.transforms.apply_trans(np.linalg.inv(x.affine), coords)).astype(int)
ras_coords = mne.transforms.apply_trans(x.header.get_vox2ras_tkr(), vox_coords)
ras_coords = ras_coords / 1000

montage = mne.channels.make_dig_montage(ch_pos=dict(zip(raw.ch_names, ras_coords)), coord_frame='mri')
raw.set_montage(montage)

/tmp/ipykernel_3730404/702744162.py:7: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw.set_montage(montage)


<RawBrainVision | sub-01_ses-iemu_task-film_acq-clinical_run-1_ieeg.eeg, 103 x 860254 (420.0 s), ~147 kB, data not loaded>

### Visualize electrodes

*By default, if SEEG channels are present, MNE will make the visualization transparent

In [8]:
from packaging import version

if version.parse(mne.__version__) <= version.parse('0.22.0'):
    fig = mne.viz.plot_alignment(raw.info,
                   subject='sub-' + subject,
                   subjects_dir=os.path.dirname(fs_dir),
                   surfaces=['pial'],
                   coord_frame='mri')
else:
    # trans argument became mandatory
    identity_trans = mne.transforms.Transform('head', 'mri')
    fig = mne.viz.plot_alignment(raw.info, trans=identity_trans,
               subject='sub-' + subject,
               subjects_dir=os.path.dirname(fs_dir),
               surfaces=['pial'],
               coord_frame='mri')
mne.viz.set_3d_view(fig, 180, 70, distance=.5)

Channel types::	seeg: 103
